In [2]:
library(org.Hs.eg.db)
library(AnnotationDbi)
library(ggplot2)
library(lumi)
library(missMethyl)
library(ggridges)

In [23]:
library(reshape2)

In [3]:
cobre_beta = readRDS("/nfs/dcmb-lgarmire/yhdu/COBRE Final Documents/Figure 1 Differential analysis/cobre_beta.rds")
cobre_pd = readRDS("/nfs/dcmb-lgarmire/yhdu/COBRE Final Documents/Figure 1 Differential analysis/cobre_pd.rds")
anno = readRDS("/nfs/dcmb-lgarmire/yhdu/COBRE Final Documents/Figure 1 Differential analysis/anno_450k.rds")

In [12]:
infla_genes = read.csv('infla_genes.csv',header=FALSE)

In [14]:
anno = as.data.frame(anno)

In [15]:
infla_cpg = anno[anno$UCSC_RefGene_Name%in%infla_genes$V1,]

In [18]:
infla_cpg_promoter = infla_cpg[infla_cpg$UCSC_RefGene_Group%in%c('TSS200','TSS1500'),]

In [19]:
dim(infla_cpg_promoter)
infla_cpg_promoter[1:5,]

[1] 1158    3

,Name,UCSC_RefGene_Name,UCSC_RefGene_Group
,<chr>,<chr>,<chr>
cg07816287,cg07816287,GPC3,TSS200
cg16272791,cg16272791,GPC3,TSS1500
cg23837911,cg23837911,GPC3,TSS200
cg00305374,cg00305374,F3,TSS200
cg00545681,cg00545681,F3,TSS1500


In [20]:
cobre_beta_infla = cobre_beta[rownames(cobre_beta)%in%rownames(infla_cpg_promoter),]
dim(cobre_beta_infla)

[1] 1078   72

In [41]:
cobre_beta_immune_control = cobre_beta_infla[,cobre_pd[cobre_pd$Sample_Group=='control',]$Sample_Name]
cobre_beta_immune_obese = cobre_beta_infla[,cobre_pd[cobre_pd$Sample_Group=='obese',]$Sample_Name]
control_immune_mean = colMeans(cobre_beta_immune_control)
obese_immune_mean = colMeans(cobre_beta_immune_obese)

In [44]:
control_immune_df = cbind(control_immune_mean,rep('control',length(control_immune_mean)))
colnames(control_immune_df) = c('Mean_beta','Group')
obese_immune_df = cbind(obese_immune_mean,rep('obese',length(obese_immune_mean)))
colnames(obese_immune_df) = c('Mean_beta','Group')
immune_cpg = as.data.frame(rbind(control_immune_df,obese_immune_df))
immune_cpg$Mean_beta = as.numeric(immune_cpg$Mean_beta)

In [45]:
wilcox.test(control_immune_mean,obese_immune_mean)


	Wilcoxon rank sum exact test

data:  control_immune_mean and obese_immune_mean
W = 482, p-value = 0.06492
alternative hypothesis: true location shift is not equal to 0


In [48]:
plot_inflam = ggplot(immune_cpg, aes(x = Group, y = Mean_beta, fill = Group)) +
  geom_violin(trim = FALSE) +
  labs(title = "Inflammation methylation score\n(Wilcox P = 0.0649)",
       x = "Group",
       y = "Mean Beta Value") +
  theme_minimal() +
    stat_summary(fun.data = "mean_cl_boot", geom = "crossbar",
               colour = "black", width = 0.2)

In [49]:
png('plot_inflam.png',width=4,height=3,unit='in',res=300)
plot_inflam
dev.off()

png 
  2